In [45]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [46]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [47]:
####### Data-Preprocessingg and NLP #######    
words = []
classes = []
documents = []
ignore_words = ['?','!']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [48]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))

In [49]:
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

34 documents
10 classes ['goodbye', 'greeting', 'jokes', 'jokesagain', 'morequotes', 'options', 'quotes', 'search_by_title_on_youtube', 'search_youtube', 'thanks']
46 unique lemmatized words ["'s", '1', '2', '3', 'a', 'anyone', 'are', 'be', 'bye', 'can', 'could', 'day', 'do', 'good', 'goodbye', 'hello', 'help', 'helpful', 'hi', 'how', 'is', 'joke', 'later', 'me', 'more', 'motivational', 'no', 'offered', 'on', 'please', 'provide', 'quote', 'search', 'see', 'some', 'support', 'tell', 'thank', 'thanks', 'that', 'there', 'title', 'video', 'what', 'you', 'youtube']


In [50]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [51]:
###### create our training data ######
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)


In [52]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [53]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [54]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [55]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
34/34 [==============================] - 0s 4ms/step - loss: 2.4105 - accuracy: 0.0882
Epoch 2/200
34/34 [==============================] - 0s 399us/step - loss: 2.2976 - accuracy: 0.1176
Epoch 3/200
34/34 [==============================] - 0s 380us/step - loss: 2.2562 - accuracy: 0.1765
Epoch 4/200
34/34 [==============================] - 0s 415us/step - loss: 2.1730 - accuracy: 0.2059
Epoch 5/200
34/34 [==============================] - 0s 487us/step - loss: 2.1759 - accuracy: 0.1471
Epoch 6/200
34/34 [==============================] - 0s 407us/step - loss: 2.0913 - accuracy: 0.2353
Epoch 7/200
34/34 [==============================] - 0s 406us/step - loss: 2.0687 - accuracy: 0.3824
Epoch 8/200
34/34 [==============================] - 0s 350us/step - loss: 1.9763 - accuracy: 0.3529
Epoch 9/200
34/34 [==============================] - 0s 484us/step - loss: 1.9862 - accuracy: 0.3824
Epoch 10/200
34/34 [==============================] - 0s 379us/step - loss: 1.9344 - accuracy

34/34 [==============================] - 0s 372us/step - loss: 0.1231 - accuracy: 1.0000
Epoch 82/200
34/34 [==============================] - 0s 351us/step - loss: 0.0915 - accuracy: 1.0000
Epoch 83/200
34/34 [==============================] - 0s 342us/step - loss: 0.0814 - accuracy: 0.9706
Epoch 84/200
34/34 [==============================] - 0s 358us/step - loss: 0.1630 - accuracy: 0.9412
Epoch 85/200
34/34 [==============================] - 0s 391us/step - loss: 0.0940 - accuracy: 1.0000
Epoch 86/200
34/34 [==============================] - 0s 365us/step - loss: 0.1257 - accuracy: 0.9412
Epoch 87/200
34/34 [==============================] - 0s 402us/step - loss: 0.1687 - accuracy: 0.9412
Epoch 88/200
34/34 [==============================] - 0s 364us/step - loss: 0.1351 - accuracy: 0.9412
Epoch 89/200
34/34 [==============================] - 0s 333us/step - loss: 0.0844 - accuracy: 1.0000
Epoch 90/200
34/34 [==============================] - 0s 321us/step - loss: 0.1573 - accuracy: 

34/34 [==============================] - 0s 328us/step - loss: 0.0346 - accuracy: 1.0000
Epoch 161/200
34/34 [==============================] - 0s 337us/step - loss: 0.1228 - accuracy: 0.9706
Epoch 162/200
34/34 [==============================] - 0s 345us/step - loss: 0.0654 - accuracy: 1.0000
Epoch 163/200
34/34 [==============================] - 0s 310us/step - loss: 0.1145 - accuracy: 0.9706
Epoch 164/200
34/34 [==============================] - 0s 345us/step - loss: 0.0176 - accuracy: 1.0000
Epoch 165/200
34/34 [==============================] - 0s 342us/step - loss: 0.1307 - accuracy: 0.9706
Epoch 166/200
34/34 [==============================] - 0s 307us/step - loss: 0.0558 - accuracy: 0.9706
Epoch 167/200
34/34 [==============================] - 0s 295us/step - loss: 0.0141 - accuracy: 1.0000
Epoch 168/200
34/34 [==============================] - 0s 401us/step - loss: 0.0456 - accuracy: 1.0000
Epoch 169/200
34/34 [==============================] - 0s 361us/step - loss: 0.1003 - a

In [56]:
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )